In [1]:
# Import libraries needed
import csv
import json
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date, text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

In [2]:
# made json a csv

with open('rximagesAll.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile, lines=True)

df.to_csv('rximagesAll.csv', encoding='utf-8', index=False)

In [3]:
df_top_10 = df.head(10)
df_top_10

,_id,ndc11,part,rxcui,acqDate,name,labeler,deaSchedule,attribution,mpc,...,nlmImageSize600,rxnavImageSize300,nlmImageSize300,rxnavImageSize120,nlmImageSize120,relabelersNdc9,status,splSetId,splRootId,splVersion
0,{'$oid': '5a32f687eb4525ea9e82bee3'},63459-0502-30,1,261106,09-26-2013,Fentanyl 0.2 MG Oral Lozenge [Actiq],"Cephalon, Inc.",CII,National Library of Medicine | Lister Hill Nat...,"{'shape': 'BULLET', 'size': 19, 'color': 'WHIT...",...,55054.0,26586,17477.0,4516,3643.0,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '5a32f687eb4525ea9e82bee4'},63459-0504-30,1,262219,09-26-2013,Fentanyl 0.4 MG Oral Lozenge [Actiq],"Cephalon, Inc.",CII,National Library of Medicine | Lister Hill Nat...,"{'shape': 'BULLET', 'size': 19, 'color': 'WHIT...",...,55793.0,26739,17645.0,4617,3726.0,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '5a32f687eb4525ea9e82bee5'},63459-0506-30,1,261107,09-26-2013,Fentanyl 0.6 MG Oral Lozenge [Actiq],"Cephalon, Inc.",CII,National Library of Medicine | Lister Hill Nat...,"{'shape': 'BULLET', 'size': 19, 'color': 'WHIT...",...,57579.0,27285,18274.0,4686,3895.0,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '5a32f687eb4525ea9e82bee6'},63459-0508-30,1,261108,09-26-2013,Fentanyl 0.8 MG Oral Lozenge [Actiq],"Cephalon, Inc.",CII,National Library of Medicine | Lister Hill Nat...,"{'shape': 'BULLET', 'size': 19, 'color': 'WHIT...",...,55495.0,27041,17932.0,4558,3704.0,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '5a32f687eb4525ea9e82bee7'},63459-0512-30,1,261109,09-26-2013,Fentanyl 1.2 MG Oral Lozenge [Actiq],"Cephalon, Inc.",CII,National Library of Medicine | Lister Hill Nat...,"{'shape': 'BULLET', 'size': 19, 'color': 'WHIT...",...,56360.0,26954,17943.0,4641,3766.0,NaN,NaN,NaN,NaN,NaN
5,{'$oid': '5a32f687eb4525ea9e82bee8'},63459-0516-30,1,261110,09-26-2013,Fentanyl 1.6 MG Oral Lozenge [Actiq],"Cephalon, Inc.",CII,National Library of Medicine | Lister Hill Nat...,"{'shape': 'BULLET', 'size': 19, 'color': 'WHIT...",...,56536.0,26906,17890.0,4617,3779.0,NaN,NaN,NaN,NaN,NaN
6,{'$oid': '5a32f688eb4525ea9e82bee9'},00093-0311-01,1,978006,11-05-2009,Loperamide Hydrochloride 2 MG Oral Capsule,Teva Pharmaceuticals USA Inc,NaN,National Library of Medicine | Lister Hill Nat...,"{'shape': 'CAPSULE', 'size': 14, 'color': 'BRO...",...,69417.0,30889,21618.0,6289,5556.0,"[{'@sourceNdc9': '00093-0311', 'ndc9': ['16590...",Former imprint,NaN,NaN,NaN
7,{'$oid': '5a32f688eb4525ea9e82beea'},00093-3165-01,1,197985,12-02-2009,Minocycline 50 MG Oral Capsule,Teva Pharmaceuticals USA Inc,NaN,National Library of Medicine | Lister Hill Nat...,"{'shape': 'CAPSULE', 'size': 16, 'color': 'PIN...",...,55548.0,28417,18655.0,5868,5151.0,"[{'@sourceNdc9': '00093-3165', 'ndc9': ['55154...",Former imprint,NaN,NaN,NaN
8,{'$oid': '5a32f688eb4525ea9e82beeb'},00093-0810-01,1,198045,12-02-2009,Nortriptyline 10 MG Oral Capsule,Teva Pharmaceuticals USA Inc,NaN,National Library of Medicine | Lister Hill Nat...,"{'shape': 'CAPSULE', 'size': 16, 'color': 'WHI...",...,60149.0,29305,20145.0,6225,5564.0,"[{'@sourceNdc9': '00093-0810', 'ndc9': ['58118...",Former imprint,NaN,NaN,NaN
9,{'$oid': '5a32f688eb4525ea9e82beec'},00093-0811-01,1,317136,12-02-2009,Nortriptyline 25 MG Oral Capsule,Teva Pharmaceuticals USA Inc,NaN,National Library of Medicine | Lister Hill Nat...,"{'shape': 'CAPSULE', 'size': 14, 'color': 'WHI...",...,77865.0,28872,26431.0,6067,7227.0,"[{'@sourceNdc9': '00093-0811', 'ndc9': ['33261...",Former imprint,NaN,NaN,NaN


In [4]:
df.drop(columns = ['_id', 'mpc', 'ingredients', 'relabelersNdc9'], inplace=True)

In [5]:
df.shape

(4392, 31)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4392 entries, 0 to 4391
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ndc11                 4392 non-null   object 
 1   part                  4392 non-null   int64  
 2   rxcui                 4392 non-null   int64  
 3   acqDate               4388 non-null   object 
 4   name                  4392 non-null   object 
 5   labeler               4392 non-null   object 
 6   deaSchedule           170 non-null    object 
 7   attribution           4392 non-null   object 
 8   ingredientsAvailable  4392 non-null   bool   
 9   rxnavImageObjectId    4392 non-null   int64  
 10  rxnavImageFileName    4392 non-null   object 
 11  rxnavImageSize        4392 non-null   int64  
 12  nlmImageObjectId      4392 non-null   int64  
 13  nlmImageFileName      4392 non-null   object 
 14  nlmImageSize          4392 non-null   int64  
 15  rxnavImageSizeFull   

In [7]:
engine = create_engine('sqlite:///pill_predictions.db', echo=False)

In [9]:
df.to_sql(name='users3', con=engine)

with engine.connect() as conn:
   conn.execute(text("SELECT * FROM users3")).fetchall()

In [11]:
# Create session for database
Session = sessionmaker(bind=engine)
session = Session()

In [12]:
# Close session
session.close()

In [ ]:
# Define base class for SQLAlchemy model
# Base = declarative_base()

In [ ]:
# Define structures
# class Predictions(Base):
    #__tablename__ = 'pill_combined_test'  

    # Define columns for the table
    # image_ID = Column(String, primary_key=True)
    # image_size_120 = Column(Integer)
    # image_size_300 = Column(Integer)
    # image_size_600 = Column(Integer)
    #image_size_800 = Column(Integer)
    # image_size_1024 = Column(Integer)
    # image_size_original = Column(Integer)

In [ ]:
# SQLite database path
# db_path = 'sqlite:///predictions.db'
# engine = create_engine(db_path)

In [ ]:
# Create tables
# Base.metadata.create_all(engine)

In [ ]:
# Create a session for the database
# Session = sessionmaker(bind=engine)
# session = Session()

In [ ]:
# Upload data from 'rximagesAll'
# with open('rximagesAll.csv', 'r') as file:
    # csv_reader = csv.DictReader(file)
    # for row in csv_reader:
        # data = Predictions(
            # image_ID = row['_id'],
            # image_size_120 = row["rxnavImageSize120"],
            # image_size_300 = row["rxnavImageSize300"],
            # image_size_600 = row["rxnavImageSize600"],
            # image_size_800 = row["rxnavImageSize800"],
            # image_size_1024 = row["rxnavImageSize1024"],
            # image_size_original = row["rxnavImageSizeFull"]
        # )
        
        # session.add(data)

In [ ]:
# Commit the changes to the database
# session.commit()